# Introduction

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed placerat pellentesque tortor at luctus. Cras varius dui odio, sit amet sodales ipsum ornare non. Mauris imperdiet interdum fermentum. Suspendisse ac nisl in dui feugiat pellentesque. In ac condimentum ligula. Nam nec arcu vel eros eleifend ultricies ut eu arcu. Phasellus dictum mauris a nunc tempor pellentesque vitae eget orci. Vestibulum gravida gravida ligula, eget rutrum dui pulvinar iaculis. Curabitur fermentum elementum purus, ac vulputate magna consectetur eu. Phasellus sodales facilisis tortor, nec iaculis ex aliquam a. Phasellus euismod justo a convallis tempus. Curabitur dignissim mi mauris.

Maecenas congue ut lacus ac dapibus. Maecenas mollis, sem eget egestas pulvinar, eros augue aliquam neque, id porta neque lacus a augue. Vestibulum at pharetra velit, in facilisis mauris. Aenean sollicitudin elementum mi, eget pharetra nibh vestibulum sodales. Mauris in malesuada ipsum, vitae varius metus. Vestibulum non iaculis nibh. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Phasellus semper sodales metus id commodo. Quisque tincidunt, turpis quis imperdiet sollicitudin, ante dolor imperdiet nibh, nec iaculis risus massa non libero. Donec magna risus, dignissim eu semper ac, vestibulum quis tellus. Interdum et malesuada fames ac ante ipsum primis in faucibus. Integer eu justo non justo ullamcorper cursus eget vulputate erat. Nunc auctor quam posuere, varius dui in, accumsan mi. Donec aliquet lacus vitae orci ultricies feugiat. Proin viverra, felis vel euismod rutrum, ligula risus viverra orci, id maximus nisl urna vel neque. Integer sodales velit urna, in mattis leo ornare eu.

Donec molestie eget lectus nec viverra. Nulla sed semper mauris, vitae suscipit mi. Vestibulum vel sodales magna. Vivamus laoreet vestibulum nibh, sed ornare lacus luctus id. Quisque fringilla lacus ac interdum iaculis. Aliquam accumsan nisl et libero dignissim eleifend. Sed magna enim, dictum sodales odio nec, interdum interdum tortor. Morbi sodales sem libero, in interdum diam cursus et. Quisque malesuada imperdiet sem, blandit luctus purus pharetra vel.

# Data and Methods

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
import atddm
from constants import AIRPORTS, COLORS, TZONES, CODES, BEGDT, ENDDT
import seaborn as sns
from math import log2, floor
from matplotlib import colors

In [ ]:
BEGDT = pd.Timestamp(BEGDT)
ENDDT = pd.Timestamp(ENDDT)
INTERVAL = 10
NLEVEL = 8
WTYPE = 'db5'
RYCMAP = colors.ListedColormap(['red', 'darkred',
                                'coral', 'orangered',
                                'goldenrod', 'darkgoldenrod',
                                'limegreen', 'darkgreen',
                                'lightseagreen', 'seagreen',
                                'steelblue', 'cadetblue',
                                'blue', 'navy',
                                'darkviolet', 'purple'
                               ])

In [ ]:
dd = atddm.load(subset=CODES)
m3_bin = {}
# m1_bin = {}
CODES.sort()

for code in CODES:
    indx = pd.date_range(start=BEGDT, end=ENDDT,
                         freq=str(INTERVAL)+'min', tz=TZONES[code])
    m3_bin[code] = atddm.binarrivals(dd[code].M3_FL240,
                                     interval=INTERVAL,
                                     tz=TZONES[code])[indx].fillna(0)
#    m1_bin[code] = atddm.binarrivals(dd[code].M1_FL240,
#                                     interval=INTERVAL,
#                                     tz=TZONES[code])[indx].fillna(0)

## Wavelet analysis

In [ ]:
def findm(length, n=NLEVEL):
    return floor(length/2**n)


def trimmedindex(serie, nlev=NLEVEL):
    m = findm(len(serie), nlev)
    lenmax = m * 2**nlev
    return serie.index[:lenmax]


def wvlt_analysis(serie, wtype=WTYPE, nlev=NLEVEL):
    df = pd.DataFrame(index=trimmedindex(serie, nlev))
    # df['signal'] = serie.iloc[:len(df)]
    x = serie.iloc[:len(df)]
    for j in range(nlev):
        level = j+1
        ca, cd = pywt.dwt(x, wtype, mode='per')
        x = np.copy(ca)
        for i in range(level):
            apx = pywt.idwt(ca, None, wtype, mode= 'per')
            det = pywt.idwt(None, cd, wtype, mode= 'per')
            ca = apx
            cd = det
        for lbl, vec in zip(['approx', 'detail'], [apx, det]):
            label = 'level_{:d}_{:s}'.format(level, lbl)
            df[label] = vec
    colnames = []
    for j in range(nlev):
        level = j+1
        for lbl in ['approx', 'detail']:
            label = (level, lbl)
            colnames.append(label)
    df.columns = pd.MultiIndex.from_tuples(colnames, names=['level','type'])
    df[(0, 'signal')] = serie.iloc[:len(df)]
    return df.sort_index(axis=1)

In [ ]:
# m1_wvlt = {}
m3_wvlt = {}
levels = 8
for code in CODES:
    # m1_wvlt[code] = wvlt_analysis(m1_bin[code], nlev=levels)
    m3_wvlt[code] = wvlt_analysis(m3_bin[code], nlev=levels)
titles = [('Level {:d} :: approximation'.format(i), 
           'Level {:d} :: detail'.format(i)) for i in range(1, levels+1)]
titles = [item for sublist in titles for item in sublist]

# Results

## Frankfurt

In [ ]:
f, axes = plt.subplots(levels, 2, figsize=(20,25))
df = m3_wvlt['EDDF'].loc[:, (slice(1,8), slice(None))]
df.plot(ax=axes, subplots=True, colormap=RYCMAP,
        legend=False, title=titles)
plt.tight_layout()
plt.show()

## London Heathrow

In [ ]:
f, axes = plt.subplots(levels, 2, figsize=(20,24))
df = m3_wvlt['EGLL'].loc[:, (slice(1,8), slice(None))]
df.plot(ax=axes, subplots=True, colormap=RYCMAP,
        legend=False, title=titles)
plt.tight_layout()
plt.show()

## London Gatwick

In [ ]:
f, axes = plt.subplots(levels, 2, figsize=(20,24))
df = m3_wvlt['EGKK'].loc[:, (slice(1,8), slice(None))]
df.plot(ax=axes, subplots=True, colormap=RYCMAP,
        legend=False, title=titles)
plt.tight_layout()
plt.show()

## Amsterdam Schiphol

In [ ]:
f, axes = plt.subplots(levels, 2, figsize=(20,24))
df = m3_wvlt['EHAM'].loc[:, (slice(1,8), slice(None))]
df.plot(ax=axes, subplots=True, colormap=RYCMAP,
        legend=False, title=titles)
plt.tight_layout()
plt.show()

## Paris Charles de Gaulle

In [ ]:
f, axes = plt.subplots(levels, 2, figsize=(20,24))
df = m3_wvlt['LFPG'].loc[:, (slice(1,8), slice(None))]
df.plot(ax=axes, subplots=True, colormap=RYCMAP,
        legend=False, title=titles)
plt.tight_layout()
plt.show()

## Madrid-Barajas

In [ ]:
f, axes = plt.subplots(levels, 2, figsize=(20,24))
df = m3_wvlt['LEMD'].loc[:, (slice(1,8), slice(None))]
df.plot(ax=axes, subplots=True, colormap=RYCMAP,
        legend=False, title=titles)
plt.tight_layout()
plt.show()

## Rome Fiumicino

In [ ]:
f, axes = plt.subplots(levels, 2, figsize=(20,24))
df = m3_wvlt['LIRF'].loc[:, (slice(1,8), slice(None))]
df.plot(ax=axes, subplots=True, colormap=RYCMAP,
        legend=False, title=titles)
plt.tight_layout()
plt.show()

## Athens International

In [ ]:
f, axes = plt.subplots(levels, 2, figsize=(20,24))
df = m3_wvlt['LGAV'].loc[:, (slice(1,8), slice(None))]
df.plot(ax=axes, subplots=True, colormap=RYCMAP,
        legend=False, title=titles)
plt.tight_layout()
plt.show()